Transforming ver_i.csv dataframes.
Column annotator_label to annotator_paragraph:
[[ann1_pair1, ann2_pair1, ann_3_pair1, ...], [ann1_pair2, ann2_pair2, ann3_pair3, ...]]

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
paragraphs_gold = pd.read_csv('./all_paragraphs.csv') 

In [3]:
paragraphs_gold

,text-id,text,label,version
0,medium-17,"Due to the overreach on the initial layoffs, c...",1,3
1,medium-17,I used to be in Pharmacy. Then I did a researc...,0,3
2,medium-17,So if a medical center has any that are partia...,1,3
3,medium-17,Open vials are supposed to be discarded after ...,1,3
4,medium-17,It costs about 15-35 cents to make a mL of ins...,1,3
...,...,...,...,...
166,hard-252,We don’t make more money today as land develop...,0,-1
167,hard-252,"Blame red tape, environmental “protection”, an...",1,-1
168,hard-252,If you want the government to intervene to mak...,1,-1
169,hard-252,Most politicians in my province (and probably ...,0,-1


In [4]:
dataframes = []
for i in range(1,9):
    dataframes.append(pd.read_csv(f'./ver_{i}.csv'))

def parse_array_string(s):
    # Use regex to find all occurrences of numbers inside array([...])
    array_strings = re.findall(r'array\(\[([0-9, ]+)\]', s)
    # Convert found strings to numpy arrays
    arrays = [np.fromstring(arr, sep=', ', dtype=int) for arr in array_strings]
    return arrays

for j in range(len(dataframes)):
    df = dataframes[j]
    for i in range(len(df)):
        s = df['annotator_label'][i]
        # Extract the numpy arrays from the string
        arrays = parse_array_string(s)

        # Convert to a numpy array of numpy arrays
        result = arrays
        df['annotator_label'][i] = result

In [5]:
print(dataframes[7])

      text-id                                    annotator_label
0    hard-118  [[1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 0...
1    hard-762  [[0, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1...
2    hard-693  [[1, 1, 0, 0], [0, 1, 1, 0], [0, 0, 0, 1], [1,...
3  medium-253  [[1, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 1, 1, 1...


In [6]:
# transforming labels so that its a big array of arrays of labels for each pair of paragraph for each annotator
# [[ann1_pair1, ann2_pair1, ann_3_pair1, ...], [ann1_pair2, ann2_pair2, ann3_pair3, ...]]

def parsing_dataframe(df):
    transformed_data = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        text_id = row['text-id']
        labels = row['annotator_label']

        # Flatten the list of lists
        flattened_labels = [item for sublist in labels for item in sublist]

        # Reshape the flattened list into separate lists for each position
        reshaped_labels = [flattened_labels[i::len(labels[0])] for i in range(len(labels[0]))]

        # Create a new dictionary with the transformed labels
        transformed_row = {'text-id': text_id, 'annotator_paragraph': reshaped_labels}

        # Append the transformed row to the list
        transformed_data.append(transformed_row)

    # Convert the list of dictionaries back into a DataFrame
    transformed_df = pd.DataFrame(transformed_data)

    return transformed_df

print(dataframes[0]['annotator_label'][0])
parsing_dataframe(dataframes[0])['annotator_paragraph'][0]

[array([1, 0, 1]), array([1, 1, 1]), array([1, 1, 1]), array([1, 0, 1]), array([1, 1, 1]), array([0, 1, 0]), array([1, 1, 1]), array([1, 1, 0]), array([0, 1, 0]), array([1, 1, 0])]


[[1, 1, 1, 1, 1, 0, 1, 1, 0, 1],
 [0, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 1, 0, 0, 0]]

In [7]:
for i in range(8):
    dataframes[i] = parsing_dataframe(dataframes[i])

In [8]:
for i in range(8):
    dataframes[i].to_csv(f'./paragraphs_labels/ver_{i+1}.csv', index=False)